In [0]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

import os
cwd_path = os.getcwd()+'/drive/My Drive/Colab Notebooks/IRLAB/IR_PROJECT'
os.chdir(cwd_path)
os.getcwd()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


'/content/drive/My Drive/Colab Notebooks/IRLAB/IR_PROJECT'

In [0]:
# important imports
from bs4 import BeautifulSoup
import numpy as np
from collections import Counter

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy as sc
from scipy.sparse.linalg import svds

from collections import OrderedDict
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
from nltk.corpus import wordnet
!python -m nltk.downloader wordnet
!python -m spacy download en_core_web_lg
!pip3 install spacy-wordnet
import spacy 
nlp = spacy.load("en_core_web_sm") 


/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
     |████████████████████████████████| 827.9MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180944 sha256=5f9ae2325c03e2f2d9f7320362c78029cb5011c0c1d1bcdced46c20eeb58e7e3
  Stored in directory: /tmp/pip-ephem-wheel-cache-4ivvq9ek/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
     |████████████████████████████████| 655kB 2.8MB/s 
     |████████████████████████████████| 1.4MB 14.8MB/s 
  Created wheel for spacy-wordnet: filename=spacy_wordnet-0

In [0]:
# function to extract data 
def extract_data(filepath, tag):
    """ Function to extract data query text from file """
    
    with open(filepath, "r") as file:
        filedata = file.read()

    soup = BeautifulSoup(filedata, "lxml")
    query_list = soup.find_all(tag)

    return query_list

In [0]:
# funtion for preprocessing and text cleaning
import string
!pip3 install contractions
import contractions
import re
def preprocessing(data):
    """Function to preprocess the raw data"""
    
    # Removing contractions
    contractions_removed = contractions.fix(data.lower(), leftovers=True, slang=True)
    
    # Removing urls
    url_regex = r'\b((?:https?://)+(?:(?:www\.)?(?:[\da-z\.-]+)\.(?:[a-z]{2,6})|(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)|(?:(?:[0-9a-fA-F]{1,4}:){7,7}[0-9a-fA-F]{1,4}|(?:[0-9a-fA-F]{1,4}:){1,7}:|(?:[0-9a-fA-F]{1,4}:){1,6}:[0-9a-fA-F]{1,4}|(?:[0-9a-fA-F]{1,4}:){1,5}(?::[0-9a-fA-F]{1,4}){1,2}|(?:[0-9a-fA-F]{1,4}:){1,4}(?::[0-9a-fA-F]{1,4}){1,3}|(?:[0-9a-fA-F]{1,4}:){1,3}(?::[0-9a-fA-F]{1,4}){1,4}|(?:[0-9a-fA-F]{1,4}:){1,2}(?::[0-9a-fA-F]{1,4}){1,5}|[0-9a-fA-F]{1,4}:(?:(?::[0-9a-fA-F]{1,4}){1,6})|:(?:(?::[0-9a-fA-F]{1,4}){1,7}|:)|fe80:(?::[0-9a-fA-F]{0,4}){0,4}%[0-9a-zA-Z]{1,}|::(?:ffff(?::0{1,4}){0,1}:){0,1}(?:(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])|(?:[0-9a-fA-F]{1,4}:){1,4}:(?:(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])))(?::[0-9]{1,4}|[1-5][0-9]{4}|6[0-4][0-9]{3}|65[0-4][0-9]{2}|655[0-2][0-9]|6553[0-5])?(?:/[\w\.-]*)*/?)\b'
    urls_extracted = re.findall(url_regex, contractions_removed)
    contractions_removed = re.sub(url_regex, " ", contractions_removed, flags=re.MULTILINE)
    
    # Removing punctuations
    punc_to_be_removed = string.punctuation
    punctuation_removed = ''.join([char for char in contractions_removed if char not in punc_to_be_removed])
    
    # Removing multiple whitespace
    multi_whitespace_removed = re.sub(r'\s+', ' ', punctuation_removed)
    
    # Tokenizating
    tokens = word_tokenize(multi_whitespace_removed)

    return tokens

     |████████████████████████████████| 317kB 3.8MB/s 
     |████████████████████████████████| 245kB 43.6MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81699 sha256=827a3fd431745baf38453af6f7570d52cb189eb50627a2b2056c0bad9ad7a52e
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [0]:
# getting into current working directory
cwd_path = os.getcwd()

main_dir_name = 'FIRE_Dataset_EN_2010'
all_queries_file_name = 'en.topics.76-125.2010.txt'
main_corpus_dir_name = 'TELEGRAPH_UTF8'

main_dir_path = cwd_path + '/' + main_dir_name

In [0]:
# creating query list
all_queries_file_path = main_dir_path + '/' + all_queries_file_name
query_list = extract_data(all_queries_file_path, 'desc')
query_list = [query.get_text().lower() for query in query_list]
pickle.dump(query_list,open("query_list.txt",'wb'))

In [0]:
# print(len(query_list))

50


In [0]:
 list_of_queryterm_lists=[preprocessing(query) for query in query_list]

# list_of_queryterm_lists[0]

['reasons',
 'behind',
 'the',
 'protests',
 'by',
 'meena',
 'leaders',
 'against',
 'the',
 'inclusion',
 'of',
 'gurjars',
 'in',
 'the',
 'scheduled',
 'tribes']

In [0]:
# print(len(list_of_queryterm_lists))
# print(len(query_list))

50
50


In [0]:
pickle.dump(list_of_queryterm_lists,open("list_of_queryterm_lists.txt","wb"))

In [0]:
# entity extraction from top 10 relevent results of elasticsearch and 
# then expanding query with synonyms of these entities
top_10_relevant_list=pickle.load(open("top_10_relevant_list.txt","rb"))
for idx,doc_list in enumerate(top_10_relevant_list):
  print(idx)
  for doc_data in doc_list:
    doc=nlp(doc_data)
    temp=list()
    for token in doc:
      if token.pos_ == 'NOUN':
        temp.append(token.text)
        synonyms_3 = list()
        count = 0
        for syn in wordnet.synsets(token.text):
          for l in syn.lemmas():                               # BREAK THE LOOP IF COUNT == 3
            if count < 3:
              synonyms_3.append(l.name())
            count += 1
        temp.extend(synonyms_3)
    query_list[idx]+=" "
    query_list[idx]+=' '.join(temp)
    list_of_queryterm_lists[idx]+=(temp)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [0]:
# query_list[0]

"\nreasons behind the protests by meena leaders against the\ninclusion of gurjars in the scheduled tribes.\n calcutta Kolkata Calcutta student student pupil educatee rebels Rebel Reb Johnny_Reb staff staff staff staff reporter reporter newsman newsperson students student pupil educatee groups group grouping group teeth dentition teeth tooth weapons weapon arm weapon_system campaign political_campaign campaign run tribe tribe folk tribe status status position condition state state province state government government authorities regime militants militant activist pressure pressure pressure_level force_per_unit_area group group grouping group dispur evening evening eve even communitys community community community demand demand demand requirement status status position condition statement statement argument statement parliament parliament fantan sevens generals general full_general general reservations reservation reserve reservation list list listing tilt exercise exercise exercising ph

In [0]:
# (list_of_queryterm_lists[0])

['reasons',
 'behind',
 'the',
 'protests',
 'by',
 'meena',
 'leaders',
 'against',
 'the',
 'inclusion',
 'of',
 'gurjars',
 'in',
 'the',
 'scheduled',
 'tribes',
 'calcutta',
 'Kolkata',
 'Calcutta',
 'student',
 'student',
 'pupil',
 'educatee',
 'rebels',
 'Rebel',
 'Reb',
 'Johnny_Reb',
 'staff',
 'staff',
 'staff',
 'staff',
 'reporter',
 'reporter',
 'newsman',
 'newsperson',
 'students',
 'student',
 'pupil',
 'educatee',
 'groups',
 'group',
 'grouping',
 'group',
 'teeth',
 'dentition',
 'teeth',
 'tooth',
 'weapons',
 'weapon',
 'arm',
 'weapon_system',
 'campaign',
 'political_campaign',
 'campaign',
 'run',
 'tribe',
 'tribe',
 'folk',
 'tribe',
 'status',
 'status',
 'position',
 'condition',
 'state',
 'state',
 'province',
 'state',
 'government',
 'government',
 'authorities',
 'regime',
 'militants',
 'militant',
 'activist',
 'pressure',
 'pressure',
 'pressure_level',
 'force_per_unit_area',
 'group',
 'group',
 'grouping',
 'group',
 'dispur',
 'evening',
 'eveni

In [0]:
pickle.dump(list_of_queryterm_lists,open("list_of_queryterm_lists_qe2.txt","wb"))

In [0]:
pickle.dump(query_list, open('query_list_qe2.txt', 'wb'))

**TFIDF**

In [0]:
file_data_dict = pickle.load(open('file_data_dict.txt', 'rb'))
file_data_list = list(zip(*file_data_dict.values()))[0]
del file_data_dict

In [0]:
vectorizer = TfidfVectorizer(use_idf=True)
doc_vecs2 = vectorizer.fit_transform(file_data_list)

In [0]:
query_list_qe = pickle.load(open('query_list_qe2.txt', 'rb'))

In [0]:
query_vecs2 = vectorizer.transform(query_list_qe)
query_vectors = query_vecs2.toarray()

In [0]:
pickle.dump(doc_vecs2, open('doc_vecs_tfidf2.txt', 'wb'))
pickle.dump(query_vecs2, open('query_vecs_tfidf2.txt', 'wb'))

In [0]:
# print(len(query_list))
# print(len(query_list_qe))
# print(doc_vecs2.shape)
# print(query_vecs2.shape)

50
50
(125586, 250667)
(50, 250667)


In [0]:

query_document_file_name = 'en.qrels.76-125.2010.txt'
query_document_file_path = main_dir_path + '/' + query_document_file_name

with open(query_document_file_path, 'r') as qdfile:
    query_document_relation_list = qdfile.read().split('\n')
    

In [0]:
qdrow_list =  [qdrow.split() for qdrow in query_document_relation_list]
print(qdrow_list)

[['76', 'Q0', '1040901_nation_story_3702283.utf8', '0'], ['76', 'Q0', '1040901_opinion_story_3675790.utf8', '0'], ['76', 'Q0', '1040902_nation_story_3707291.utf8', '0'], ['76', 'Q0', '1040904_opinion_story_3713095.utf8', '0'], ['76', 'Q0', '1040908_calcutta_story_3729202.utf8', '0'], ['76', 'Q0', '1040908_nation_story_3730975.utf8', '0'], ['76', 'Q0', '1040910_bengal_story_3740572.utf8', '0'], ['76', 'Q0', '1040912_nation_story_3748456.utf8', '0'], ['76', 'Q0', '1040914_opinion_story_3737741.utf8', '0'], ['76', 'Q0', '1040917_frontpage_story_3767714.utf8', '0'], ['76', 'Q0', '1040918_calcutta_story_3773878.utf8', '0'], ['76', 'Q0', '1040923_foreign_story_3792490.utf8', '0'], ['76', 'Q0', '1040924_opinion_story_3794162.utf8', '0'], ['76', 'Q0', '1041006_nation_story_3846209.utf8', '0'], ['76', 'Q0', '1041006_nation_story_3846688.utf8', '0'], ['76', 'Q0', '1041006_nation_story_3848203.utf8', '0'], ['76', 'Q0', '1041007_foreign_story_3851950.utf8', '0'], ['76', 'Q0', '1041008_nation_story

In [0]:
qd_rel_list = list()


for qdrow in qdrow_list:
    try:
    
        key = int(qdrow[0])
        filename = qdrow[2]
        relevance = int(qdrow[3])
        
        if relevance == 1:
            t = (key, filename)
            qd_rel_list.append(t)

    except Exception as e:
        pass

print(*qd_rel_list, sep='\n')

(76, '1070530_nation_story_7849973.utf8')
(76, '1070602_nation_story_7865940.utf8')
(76, '1070603_nation_story_7869357.utf8')
(76, '1070611_nation_story_7906812.utf8')
(77, '1050110_foreign_story_4234881.utf8')
(77, '1060714_foreign_story_6477224.utf8')
(77, '1060715_foreign_story_6482002.utf8')
(77, '1060716_foreign_story_6485226.utf8')
(77, '1060717_foreign_story_6488684.utf8')
(77, '1060719_foreign_story_6496938.utf8')
(77, '1060720_foreign_story_6501311.utf8')
(77, '1060720_frontpage_story_6501523.utf8')
(77, '1060721_foreign_story_6506359.utf8')
(77, '1060727_foreign_index.utf8')
(77, '1060727_foreign_story_6531510.utf8')
(77, '1060802_foreign_story_6557944.utf8')
(77, '1060804_foreign_index.utf8')
(77, '1060804_foreign_story_6567126.utf8')
(77, '1060805_foreign_story_6571528.utf8')
(77, '1060806_foreign_story_6575214.utf8')
(77, '1060807_foreign_story_6578452.utf8')
(77, '1060809_foreign_story_6588005.utf8')
(77, '1060810_foreign_story_6592351.utf8')
(77, '1060814_foreign_story_6

In [0]:
# cosine similarity calculation
list_cosine_val_list=list()
def top5_doc_list(q_vecs, d_vecs, i, rep='w2v'):

  top_five_list = list()
  
  # i = 76

  for q_vec in q_vecs:
    if rep == 'w2v':
      cos_vals = cosine_similarity([q_vec], d_vecs)
    elif rep == 'tfidf':
      cos_vals = cosine_similarity(q_vec, d_vecs)
    l1 = list(enumerate(cos_vals[0]))
    l2 = sorted(l1, key=lambda x: x[1], reverse=True)
    l3=list(map(lambda x:x[1],l1))
    list_cosine_val_list.append(l3)
    top_5 = list()

    for idx, val in l2[0:10]:    
      top_5.append((file_name_list[idx], i))

    i += 1
    top_five_list.append(top_5)

  return top_five_list

**Rocchio** **Algo**

In [0]:
file_data_dict = pickle.load(open('file_data_dict.txt', 'rb'))
file_name_list = list(zip(*file_data_dict.values()))[1]
rel_doc_list = pickle.load(open('rel_doc_list.txt', 'rb'))
beta = 0.75
gamma2 = 0.15

def rocchio_algorithm2(beta, gamma, doc_vecs, query_vecs, alpha=1, rep='tfidf'):

  global rel_doc_list
  global file_name_list

  roc_query_vecs = list()

  doc_vecs_sum = sum(doc_vecs)

  for idx, query_vec in enumerate(query_vecs): # query_vec == alpha*q; where alpha == 1

    doc_idx_rel_count = len(rel_doc_list[idx])
    doc_idx_irrel_count = len(file_data_list) - doc_idx_rel_count
    if(len(rel_doc_list[idx])):
      rel_doc_names = list(zip(*rel_doc_list[idx]))[0]

    rel_doc_vecs_sum = 0

    for doc in rel_doc_names:
      rel_doc_idx = file_name_list.index(doc)
      rel_doc_vecs_sum += doc_vecs[rel_doc_idx]

    if doc_idx_rel_count != 0:
      beta_term_coeff = beta / doc_idx_rel_count
    else:
      beta_term_coeff = 1
   

    beta_term = beta_term_coeff * rel_doc_vecs_sum
    
    gamma_term_coeff = gamma / doc_idx_irrel_count
    gamma_term = 0

    irrel_doc_vecs_sum = doc_vecs_sum - rel_doc_vecs_sum

    if gamma != 0:
      gamma_term = gamma_term * irrel_doc_vecs_sum
    
    roc_query_vecs.append(query_vec + beta_term + gamma_term)
    

  return roc_query_vecs

In [0]:
file_data_dict = pickle.load(open('file_data_dict.txt', 'rb'))
file_data_list = list(zip(*file_data_dict.values()))[0]

In [0]:
roc_query_vecs_gammaPt15_tfidf = rocchio_algorithm2(beta, gamma2, doc_vecs2, query_vecs2, rep='tfidf')

In [0]:
pickle.dump(roc_query_vecs_gammaPt15_tfidf, open('roc_query_vecs_gammaPt15_tfidf2.txt', 'wb'))
# roc_query_vecs_gammaPt15_tfidf = pickle.load(open('roc_query_vecs_gammaPt15_tfidf.txt', 'rb'))

In [0]:
top_five_list_tfidf_gammaPt15 = top5_doc_list(roc_query_vecs_gammaPt15_tfidf, doc_vecs2, 76, 'tfidf')

In [0]:
print(*top_five_list_tfidf_gammaPt15,sep='\n')

[('1070603_nation_story_7869357.utf8', 76), ('1070611_nation_story_7906812.utf8', 76), ('1070602_nation_story_7865940.utf8', 76), ('1070530_nation_story_7849973.utf8', 76), ('1070605_frontpage_story_7877876.utf8', 76), ('1070602_nation_story_7865944.utf8', 76), ('1070607_opinion_story_7886010.utf8', 76), ('1070604_frontpage_story_7872841.utf8', 76), ('1070604_opinion_story_7852206.utf8', 76), ('1070531_frontpage_story_7855182.utf8', 76)]
[('1060719_foreign_story_6496938.utf8', 77), ('1060806_foreign_story_6575214.utf8', 77), ('1060814_foreign_story_6609200.utf8', 77), ('1060718_foreign_story_6491617.utf8', 77), ('1060724_foreign_story_6517899.utf8', 77), ('1060801_foreign_story_6553369.utf8', 77), ('1060727_foreign_story_6531510.utf8', 77), ('1060720_foreign_story_6501311.utf8', 77), ('1060715_foreign_story_6482002.utf8', 77), ('1060717_foreign_story_6488684.utf8', 77)]
[('1050709_nation_story_4968974.utf8', 78), ('1041105_nation_story_3968019.utf8', 78), ('1041102_nation_story_3954724

In [0]:
print(list_cosine_val_list[0])

In [0]:
def fun(d,q_no):
  for q in qd_rel_list:
    if q[0]==q_no:
      if d==q[1]:
        return 1
  return 0

In [0]:
list_relevant_list=list()
q_no=76
for i in range(50):
  print(i)
  temp=list()
  for d in file_name_list:
      temp.append(fun(d,q_no))
  list_relevant_list.append(temp)
  q_no+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [0]:
pickle.dump(list_relevant_list,open("listOfrelevantDocsList.txt","wb"))

In [0]:
print(len(list_cosine_val_list))

50


In [0]:
# calculating mean average precision
import numpy as np
from sklearn.metrics import average_precision_score
AP_list=list()
for i in range(50):
  y_true=np.array(list_relevant_list[i])
  y_score=np.array(list_cosine_val_list[i])
  AP_list.append(average_precision_score(y_true,y_score))



In [0]:
print(*AP_list,sep="\n")

1.0
0.6417768675634765
0.7435897435897436
0.5010141987829615
0.9464285714285714
0.8349673202614378
0.39330463504188606
0.33642317872384825
1.0
0.5632601215923961
0.1623549687459462
0.44085966568236074
0.22448117347194085
0.6488514222987876
0.7658119658119658
0.6575165575165576
0.5642331403452311
0.6010029615848111
0.9044720342798146
0.7848641953657629
0.6666666666666666
0.2872633145915392
0.8999999999999999
0.4738562091503268
0.747144141843975
0.4521643744710793
0.31278872157396564
0.7165771571786608
0.4921489448051948
0.7816093185774036
0.95
0.8353152820894755
0.18963972617443253
0.4290170501077889
0.7156039594241842
0.5784736652645852
0.8446261893832226
0.1922234950508065
0.9583333333333333
0.6127430973699785
0.9651515151515152
5.922767116796968e-05
0.5647677878814475
0.7798185941043083
0.8508070905129729
0.5612551846316793
0.4592195447517054
0.47319609384545946
0.29317316371931723
0.5474315769210947


In [0]:
mean_average_precision=sum(AP_list)/50

In [0]:
print(mean_average_precision)

0.6069257429666957
